# Juliaによる非線形方程式の全解探索アルゴリズムの実装

## 全解探索アルゴリズムのソースコード

In [1]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function orig_is_subset(X,Y)
    for i = 1:size(X)[1]
        if !( issubset(X[i],Y[i]) )
            return false
        end
    end
    return true
end

function orig_intersect(X,Y)
    L = X*1
    for i = 1:size(X)[1]
        L[i]= intersect(X[i],Y[i])
    end
    return L
end

function orig_is_empty(X,Y)
    for i = 1:size(X)[1]
        if ( isempty(intersect(X[i],Y[i])) )
            return true
        end
    end
    return false
end

function orig_devide(X)
    #最も長い辺を二等分する
    max_diam=0
    max_index=0
    min_diam=diam(X[1])
    min_index=1
    X1,X2=[],[]
    for (i,x) in enumerate(X)
        if diam(x)>max_diam
            max_diam=diam(x)
            max_index=i
        end
        if diam(x)<min_diam
            min_diam=diam(x)
            min_index=i
        end
    end

    if diam(X[min_index])<10^(-8)
        return false
    end
    
    X1=X*1 #値をコピーする
    X2=X*1 #値をコピーする
    X1[max_index]= intersect(X[max_index],X[max_index]-radius(X[max_index]))
    X2[max_index]= intersect(X[max_index],X[max_index]+(diam(X[max_index])-radius(X[max_index])))
    return X1,X2
end

function border(K,I)
    c=0.9
    K_width=diam.(K)
    I_width=diam.(I)
    max=0
    for i=1:size(K)[1]
        if (K_width[i]/I_width[i])>max
            max=K_width[i]/I_width[i]
        end
    end
    if max<=0.9
        return true
    else
        return false
    end
end

function remove_duplication(S)
    tmp_index=[]
    new_s=[]
    for i=1:size(S)[1]-1
        for j = i+1:size(S)[1]
            if !orig_is_empty(S[i],S[j])
                
                if (i in tmp_index) || (j in tmp_index)
                    push!(tmp_index,i)
                    push!(tmp_index,j)
                    continue
                end
                push!(tmp_index,i)
                push!(tmp_index,j)
                I = orig_intersect(S[i],S[j])
                push!(new_s,I)
            end
        end
    end
    for i=1:size(S)[1]
        if !(i in tmp_index)
            push!(new_s,S[i])
        end
    end
    
    return new_s
end


function allsol(F,X)
    dimention = size(X)[1]
    
    L = [X] #調査する区間のリスト
    S = Array[] #見つかった解のリスト
    
    count_ne_test=0
    
    
    while (size(L)[1] != 0)
        count_ne_test+=1
        
        X = popfirst!(L) #Lから区間を一つ取り出す
        
        #非存在判定
        if !( orig_is_subset([(0. .. 0.) for i=1:dimention], F(X)) ) 
            continue
        end
        
        c = mid.(X)
        j = ForwardDiff.jacobian(F,X)
        
        #非存在判定
        if !( orig_is_subset([(0. .. 0.) for i=1:dimention], F(c)+j*(X-c)) )
            continue
        end
        
        R = []
        #逆行列計算
        try
            R = inv(ForwardDiff.jacobian(F,c))
        catch e
            #逆行列計算に失敗したら区間を分割
            try
                X1,X2 = orig_devide(X)
                push!(L, X1)
                push!(L, X2)
            catch 
                continue
            end
            continue
        end
        
        M = Matrix{Float64}(I,size(R))-R*j
        K = c -R*F(c)+M*(X-c)
        
        #解なし
        if (orig_is_empty(K,X))
            continue 
        end
        
        #解あり
        if (orig_is_subset(K,X))　
            push!(S, K) #解のリストに追加
            continue
        end
        
        # 境界に乗ってた時
        if border(K,X)
            push!(L,K)
            continue
        end
        
        #分割して候補者区間に追加
        try
            X1,X2 = orig_devide(orig_intersect(K,X))
            #K and X を2分割
            push!(L, X1)
            push!(L, X2)
            continue
        catch
            continue
        end
    end
    
    #重複を排除
    S = remove_duplication(S)
    
    #区間を狭める
    for i =1:size(S)[1]
        si=S[i]
        while(maximum(radius,si) >= 10^-8)
            j=0
            try
                j = ForwardDiff.jacobian(F,si)
            catch
                break
            end
            c = mid.(si)
            R=0
            try 
                R = inv(ForwardDiff.jacobian(F,c)) 
            catch
                break
            end
            M = Matrix{Float64}(I,size(R))-R*j
            k = c -R*F(c)+M*(si-c)
            if orig_is_subset(k,si)
                si=k
            else
                break
            end
        end
        S[i]=si
    end
    
    setformat(:full)
    print("ne_test: ")
    println(count_ne_test)
    for s=1:size(S)[1] 
        print("解");print(s);print(": ");
        for i in S[s]
            print(i)
            print(" ")
        end
        println("")
    end
    
    return S
end

# 重解　＝＞区間幅に制限をつける
# 高速化？

allsol (generic function with 1 method)

## 検証0

In [2]:
X = [(-1000. ..1000.),(-1000. .. 1000.)]

f((x1, x2)) = 2*x1^3 + 2*x1*x2 - 22*x1+ x2^2 + 10 + @interval(pi)
g((x1, x2)) = x1^2 + 2*x1*sin(x2) + 2*x2^3 - 14*x2 + @interval(9.1)

F1((x,y))=[f((x,y));g((x,y))]
# runtime = @benchmark allsol(F1,X)
# @show runtime
allsol(F1,X)

ne_test: 302
解1: Interval(-4.144615044406719, -4.144615044403576) Interval(-3.2881319296964624, -3.2881319296946954) 
解2: Interval(3.257405642421258, 3.257405643262303) Interval(-3.1841244453837674, -3.1841244385652567) 
解3: Interval(0.8200219746141553, 0.820021974642201) Interval(-2.933905171600753, -2.933905171527188) 
解4: Interval(1.0843602587813665, 1.0843602587813803) Interval(1.9717987705796565, 1.9717987705796736) 
解5: Interval(-3.413099708210815, -3.413099708201609) Interval(1.6985839371054974, 1.6985839372303102) 
解6: Interval(0.720303081325082, 0.7203030813250845) Interval(0.8533781327368397, 0.8533781327368465) 
解7: Interval(-3.434514936485655, -3.4345149364856535) Interval(1.4044134733988092, 1.4044134733988114) 


7-element Vector{Any}:
 Interval{Float64}[Interval(-4.144615044406719, -4.144615044403576), Interval(-3.2881319296964624, -3.2881319296946954)]
 Interval{Float64}[Interval(3.257405642421258, 3.257405643262303), Interval(-3.1841244453837674, -3.1841244385652567)]
 Interval{Float64}[Interval(0.8200219746141553, 0.820021974642201), Interval(-2.933905171600753, -2.933905171527188)]
 Interval{Float64}[Interval(1.0843602587813665, 1.0843602587813803), Interval(1.9717987705796565, 1.9717987705796736)]
 Interval{Float64}[Interval(-3.413099708210815, -3.413099708201609), Interval(1.6985839371054974, 1.6985839372303102)]
 Interval{Float64}[Interval(0.720303081325082, 0.7203030813250845), Interval(0.8533781327368397, 0.8533781327368465)]
 Interval{Float64}[Interval(-3.434514936485655, -3.4345149364856535), Interval(1.4044134733988092, 1.4044134733988114)]

## 検証1

In [3]:
using BenchmarkTools

X = [(-10. ..10.),(-10. ..10.),(-10. ..10.),(-10. ..10.)]

rb = 10^4
rc= 5*10^3
vcc=-5
af=@interval(0.99)
ar=@interval(0.5)
is=@interval(10^(-9))
vt=@interval(0.053)
vs=@interval(-0.64)
gb=@interval(1. /rb)
gc=@interval(1. /rc)

a=[
    1 -ar 0 0;
    -af 1 0 0;
    0 0 1 -ar;
    0 0 -af 1
]

b(v1,v2,v3,v4)=[
    (is/af)*(exp(v1/vt)-1);
    (is/ar)*(exp(v2/vt)-1);
    (is/af)*(exp(v3/vt)-1);
    (is/ar)*(exp(v4/vt)-1)
]
c=[
    2*gb+gc -(gb+gc) -2*gb gb;
    -(gb+gc) gb+gc gb 0;
    -2*gb gb 2*gb+gc -(gb+gc);
    gb 0 -(gb+gc) gb+gc
]
v(v1,v2,v3,v4) = [v1;v2;v3;v4]
d=[
    gc*vcc;gb*vs-gc*vcc;gc*vcc;gb*vs-gc*vcc
]

F1((x1,x2,x3,x4))=a*b(x1,x2,x3,x4)+c*v(x1,x2,x3,x4)+d
# runtime = @benchmark allsol(F1,X)
# @show runtime
allsol(F1,X)

ne_test: 894200
解1: Interval(0.7199016412908966, 0.7199016412909762) Interval(-0.003335867013121626, -0.0033358670090992723) Interval(0.735512539929929, 0.7355125399299701) Interval(0.5755529349621364, 0.5755529349625033) 
解2: Interval(0.7035896316635898, 0.7035896317234862) Interval(-0.721807124566942, -0.7218071222975109) Interval(0.742316472957872, 0.742316472977706) Interval(0.6198872835609947, 0.6198872836578174) 
解3: Interval(0.7292896325636878, 0.7292896325636964) Interval(0.4714551618031478, 0.47145516180342534) Interval(0.729289632563688, 0.7292896325636964) Interval(0.47145516180315106, 0.47145516180342834) 
解4: Interval(0.7355125399299198, 0.7355125399299784) Interval(0.5755529349620547, 0.5755529349625786) Interval(0.7199016412908806, 0.7199016412909935) Interval(-0.0033358670139414038, -0.0033358670082131543) 
解5: Interval(0.7423164729657533, 0.742316472969826) Interval(0.6198872835994673, 0.6198872836193511) Interval(0.7035896316873896, 0.7035896316996825) Interval(-0.721

5-element Vector{Any}:
 Interval{Float64}[Interval(0.7199016412908966, 0.7199016412909762), Interval(-0.003335867013121626, -0.0033358670090992723), Interval(0.735512539929929, 0.7355125399299701), Interval(0.5755529349621364, 0.5755529349625033)]
 Interval{Float64}[Interval(0.7035896316635898, 0.7035896317234862), Interval(-0.721807124566942, -0.7218071222975109), Interval(0.742316472957872, 0.742316472977706), Interval(0.6198872835609947, 0.6198872836578174)]
 Interval{Float64}[Interval(0.7292896325636878, 0.7292896325636964), Interval(0.4714551618031478, 0.47145516180342534), Interval(0.729289632563688, 0.7292896325636964), Interval(0.47145516180315106, 0.47145516180342834)]
 Interval{Float64}[Interval(0.7355125399299198, 0.7355125399299784), Interval(0.5755529349620547, 0.5755529349625786), Interval(0.7199016412908806, 0.7199016412909935), Interval(-0.0033358670139414038, -0.0033358670082131543)]
 Interval{Float64}[Interval(0.7423164729657533, 0.742316472969826), Interval(0.6198872

## 検証2

In [48]:
X = [(-1000. ..1000.),(-1000. .. 1000.)]

f((x1, x2)) = x1^2 + x2^2 - 0.1
g((x1, x2)) = 2*x1^2 - x2 - 0.1

F4((x,y))=[f((x,y));g((x,y))]
allsol(F4,X)

f((x1, x2)) = x1^2 + x2^2 - (0.1 .. 0.1)
g((x1, x2)) = 2*x1^2 - x2 - (0.1 .. 0.1)

F4((x,y))=[f((x,y));g((x,y))]
allsol(F4,X)

f((x1, x2)) = x1^2 + x2^2 - @interval(0.1)
g((x1, x2)) = 2*x1^2 - x2 - @interval(0.1)

F4((x,y))=[f((x,y));g((x,y))]
allsol(F4,X)

解1: Interval(-0.30447511936535876, -0.3044751193651445) Interval(0.08541019662481308, 0.08541019662512381) 
解2: Interval(0.3044751193651445, 0.30447511936535876) Interval(0.08541019662481308, 0.08541019662512381) 
解1: Interval(-0.3044751193653588, -0.30447511936514443) Interval(0.08541019662481307, 0.08541019662512384) 
解2: Interval(0.30447511936514443, 0.3044751193653588) Interval(0.08541019662481307, 0.08541019662512384) 
解1: Interval(-0.3044751193653588, -0.30447511936514443) Interval(0.08541019662481307, 0.08541019662512384) 
解2: Interval(0.30447511936514443, 0.3044751193653588) Interval(0.08541019662481307, 0.08541019662512384) 
0.1 .. 0.1 == #= In[48]:20 =# @interval(0.1) = true


true